In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from ops import Operations
from session import run_session, restore_selection_matrixes, predict_form_sess
from data_gen import samples_generator, split_train_test
import tensorflow as tf
import pickle
from data_gen import samples_generator, split_train_test

path = '/home/user/neural_program_synthesis/models/summaries/RNN/np_mult-5ops/200_state_size-1500_num_samples-100_batch_size-0.005_learning_rate-1000.0_grad_norm-3_num_features-True_norm-True_share_state-3388_seed/'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
#reset graph so that tf dont complain about not found vars
tf.reset_default_graph()
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
# instanitae the model graph
model = RNN(cfg, ops)
#restore with given path
model_path = path+'/model'
total_loss_traind_train, softmaxes_traind_train, total_loss_traind_test, softmaxes_traind_test, total_loss_testd_train, softmaxes_testd_train, total_loss_testd_test, softmaxes_testd_test, last_softmax_state_train, last_hardmax_state_train, last_softmax_state_test, last_hardmax_state_test = restore_selection_matrixes(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'W:0' shape=(203, 200) dtype=float64_ref>, <tf.Variable 'b:0' shape=(200,) dtype=float64_ref>, <tf.Variable 'W2:0' shape=(200, 3) dtype=float64_ref>, <tf.Variable 'b2:0' shape=(3,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_0:0' shape=(203, 200) dtype=float64>, <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(200,) dtype=float64>, <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(200, 3) dtype=float64>, <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(3,) dtype=float64>]
num batches train: 10
num batches test: 4
INFO:tensorflow:Restoring parameters from /home/user/neural_program_synthesis/models/summaries/RNN/np_mult-5ops/200_state_size-1500_num_samples-100_batch_size-0.005_learning_rate-1000.0_grad_norm-3_num_features-True_norm-True_share_state-3388_seed//mod

In [2]:
def print_errors(error_lst, name):
    print("")
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [3]:
def print_matrix(total_error, matrix_lst):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
       

In [4]:
print_errors(total_loss_traind_train, "sofmax_train_error")
print_errors(total_loss_traind_test, "hardmax_train_error")
print_errors(total_loss_testd_train, "softmax_test_error")
print_errors(total_loss_testd_test, "hardmax_test_error")


sofmax_train_error[0] err is 2.71937637159e-06
sofmax_train_error[1] err is 2.17653786887e-08
sofmax_train_error[2] err is 3.13905000463e-09
sofmax_train_error[3] err is 4.63081495882e-08
sofmax_train_error[4] err is 6.47408677682e-08
sofmax_train_error[5] err is 7.31552021061e-08
sofmax_train_error[6] err is 4.42319412278e-08
sofmax_train_error[7] err is 1.14407360572e-07
sofmax_train_error[8] err is 8.62211731218e-08
sofmax_train_error[9] err is 5.29686961484e-08

hardmax_train_error[0] err is 0.0
hardmax_train_error[1] err is 0.0
hardmax_train_error[2] err is 0.0
hardmax_train_error[3] err is 0.0
hardmax_train_error[4] err is 0.0
hardmax_train_error[5] err is 0.0
hardmax_train_error[6] err is 0.0
hardmax_train_error[7] err is 0.0
hardmax_train_error[8] err is 0.0
hardmax_train_error[9] err is 0.0

softmax_test_error[0] err is 0.000316839252568
softmax_test_error[1] err is 2.84479345675e-06
softmax_test_error[2] err is 0.023247857207
softmax_test_error[3] err is 1.24712998939e-07

h

In [20]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res = predict_form_sess(model, cfg, x_test[0:100,], last_hardmax_state_train, model_path, mode="hard")

INFO:tensorflow:Restoring parameters from /home/user/neural_program_synthesis/models/summaries/RNN/np_mult-5ops/200_state_size-1500_num_samples-100_batch_size-0.005_learning_rate-1000.0_grad_norm-3_num_features-True_norm-True_share_state-3388_seed//model/-3000


In [21]:
res - y_test[0:100,]

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],


In [23]:
y_test[0:100,]

array([[  2.67086128e+04,   0.00000000e+00,   0.00000000e+00],
       [  4.88345904e+04,   0.00000000e+00,   0.00000000e+00],
       [ -1.13487504e+05,   0.00000000e+00,   0.00000000e+00],
       [ -1.40706137e+04,   0.00000000e+00,   0.00000000e+00],
       [  2.20260487e+05,   0.00000000e+00,   0.00000000e+00],
       [  3.33975782e+04,   0.00000000e+00,   0.00000000e+00],
       [ -3.65086764e+03,   0.00000000e+00,   0.00000000e+00],
       [  1.57406283e+04,   0.00000000e+00,   0.00000000e+00],
       [  3.27764124e+05,   0.00000000e+00,   0.00000000e+00],
       [  3.47607486e+04,   0.00000000e+00,   0.00000000e+00],
       [  1.17412119e+05,   0.00000000e+00,   0.00000000e+00],
       [ -2.87988102e+05,   0.00000000e+00,   0.00000000e+00],
       [  3.87517381e+03,   0.00000000e+00,   0.00000000e+00],
       [ -1.93399115e+03,   0.00000000e+00,   0.00000000e+00],
       [  3.42283430e+04,   0.00000000e+00,   0.00000000e+00],
       [  2.25526245e+05,   0.00000000e+00,   0.0000000

In [24]:
res

[array([[  2.67086128e+04,   0.00000000e+00,   0.00000000e+00],
        [  4.88345904e+04,   0.00000000e+00,   0.00000000e+00],
        [ -1.13487504e+05,   0.00000000e+00,   0.00000000e+00],
        [ -1.40706137e+04,   0.00000000e+00,   0.00000000e+00],
        [  2.20260487e+05,   0.00000000e+00,   0.00000000e+00],
        [  3.33975782e+04,   0.00000000e+00,   0.00000000e+00],
        [ -3.65086764e+03,   0.00000000e+00,   0.00000000e+00],
        [  1.57406283e+04,   0.00000000e+00,   0.00000000e+00],
        [  3.27764124e+05,   0.00000000e+00,   0.00000000e+00],
        [  3.47607486e+04,   0.00000000e+00,   0.00000000e+00],
        [  1.17412119e+05,   0.00000000e+00,   0.00000000e+00],
        [ -2.87988102e+05,   0.00000000e+00,   0.00000000e+00],
        [  3.87517381e+03,   0.00000000e+00,   0.00000000e+00],
        [ -1.93399115e+03,   0.00000000e+00,   0.00000000e+00],
        [  3.42283430e+04,   0.00000000e+00,   0.00000000e+00],
        [  2.25526245e+05,   0.00000000e